In [61]:
### Cleaning Mortality Trend data ###
import pandas as pd
import numpy as np
import string

covid = pd.read_csv('covid19mortality.csv')
heart = pd.read_csv('heartmortality.csv') #Year for every data observation is 2013
stroke = pd.read_csv('strokemortality.csv') #Year for every data observation is 2017

In [62]:
#Drop columns that are not necessary for analysis
covid = covid.drop(['Year','Footnote'],1)
heart = heart.drop(['Year','LocationDesc', 'DataSource','Class','Topic','Data_Value_Unit',
                   'Data_Value_Type','Data_Value_Footnote_Symbol', 'StratificationCategory1',
                   'StratificationCategory2', 'TopicID', 'LocationID','Location 1','GeographicLevel'], 1)
stroke = stroke.drop(['Year','LocationDesc','DataSource','Class','Topic','Data_Value_Unit','Data_Value_Type',
                     'StratificationCategory1','Data_Value_Footnote_Symbol'
                     ,'StratificationCategory2','TopicID','LocationID','Y_lat','X_lon','GeographicLevel'],1)

C:\Users\patas\AppData\Local\Temp/ipykernel_15400/3172309564.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  covid = covid.drop(['Year','Footnote'],1)
C:\Users\patas\AppData\Local\Temp/ipykernel_15400/3172309564.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  heart = heart.drop(['Year','LocationDesc', 'DataSource','Class','Topic','Data_Value_Unit',
C:\Users\patas\AppData\Local\Temp/ipykernel_15400/3172309564.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  stroke = stroke.drop(['Year','LocationDesc','DataSource','Class','Topic','Data_Value_Unit','Data_Value_Type',


In [63]:
#We want data analysis to be easy to process and less time to take, so let's clarify some column values and
#create a legend key to shorten the column values to look through the dataset
covid = covid.rename(columns = {'Non-Hispanic White':'W', 'Non-Hispanic Black or African American': 'B/AA',
                       'Non-Hispanic American Indian or Alaska Native':'AIAN', 'Non-Hispanic Asian':'A',
                        'Non-Hispanic Native Hawaiian or Other Pacific Islander': 'NH/OPI',
                        'Non Hispanic more than one race': 'MultipleRaces', 'Hispanic or Latino' : 'H/L'})

heart = heart.rename(columns = {'Data_Value': 'Deaths per 100,000', 'Data_Value_Footnote': 'Sufficiency?'
                               ,'Stratification1': 'Gender', 'Stratification2': 'Race/Ethnicity'})

stroke = stroke.rename(columns = {'Data_Value': 'Deaths per 100,000', 'Data_Value_Footnote': 'Sufficiency?'
                                 ,'Stratification1': 'Gender','Stratification2': 'Race/Ethnicity'})

In [64]:
#heart = heart.drop(heart.index[heart['Sufficiency?'] == 'Insufficient Data'],inplace = True)
heart = heart[heart['Sufficiency?'] != 'Insufficient Data']
stroke = stroke[stroke['Sufficiency?'] != 'Insufficient Data']

heart = heart.drop('Sufficiency?',1) #All data are now sufficient for data analysis
stroke = stroke.drop('Sufficiency?',1) #Therefore we have no use for this column
print(heart)

      LocationAbbr  Deaths per 100,000   Gender  \
0               AK               147.4  Overall   
1               AK               229.4  Overall   
2               AK               255.5  Overall   
3               AK               305.5  Overall   
5               AK               281.7  Overall   
...            ...                 ...      ...   
59068           WY               331.9     Male   
59069           WY               175.7   Female   
59073           WY               462.0  Overall   
59074           WY               586.7     Male   
59075           WY               351.3   Female   

                           Race/Ethnicity  
0                                 Overall  
1                                 Overall  
2                                 Overall  
3                                 Overall  
5                                 Overall  
...                                   ...  
59068                            Hispanic  
59069                            Hi

C:\Users\patas\AppData\Local\Temp/ipykernel_15400/2460707862.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  heart = heart.drop('Sufficiency?',1) #All data are now sufficient for data analysis
C:\Users\patas\AppData\Local\Temp/ipykernel_15400/2460707862.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  stroke = stroke.drop('Sufficiency?',1) #Therefore we have no use for this column
